In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df_train = np.load('../cache/train_p_stage1_fe2.npy')
df_test = np.load('../cache/test_p_stage1_fe2.npy')
df = pd.read_csv('../cache/stage1_labels.csv')
y = df['y'].values

df1 = pd.read_csv('../cache/stage1_p_test_labels.csv')
test_labels = df1['y'].values

wts_per_class = np.load('../cache/stage1_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()
print(wts_per_class)

{1: 4.846830985915493, 2: 6.347345132743363, 3: 36.31460674157304, 4: 3.8411078717201166, 5: 12.723140495867769, 6: 11.076363636363636, 7: 2.484784889821616, 8: 173.78947368421052, 9: 88.75675675675676}


In [3]:
y = y-1

In [10]:
wts_per_class2 = {}
wts_per_class2 = {i:wts_per_class[i+1] for i in y}

In [20]:
# SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
#        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
#        learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
#        n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
#        shuffle=True, tol=None, verbose=0, warm_start=False)
# sgd_model = SGDClassifier(class_weight=wts_per_class2)

sgd_model = SGDClassifier()

# parameters = {
#     'alpha': [0.0001, 0.0005, 0.001, 0.01, 0.1], 
#     'epsilon':[0.1, 0.5, 0.05],
#     'eta0': [0.0, 0.0001, 0.001],
#     'l1_ratio': [0.01, 0.1, 0.15, 0.2],
#     'loss': ['log'],
#     'random_state': [1337]
# }

parameters = {
    'alpha': [0.001, 0.01], 
    'max_iter': [50,250,500],
    'loss': ['log'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'power_t': [0.05, 0.1],
    'random_state': [1337],
}

p = {
    'alpha': [0.001], 
    'loss': ['log'], 
    'max_iter': [50], 
    'penalty': ['elasticnet'], 
    'power_t': [0.05], 
    'random_state': [1337]}



clf0 = GridSearchCV(sgd_model, p, n_jobs=1, 
                   cv=2,
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)



In [21]:
clf0.fit(df_train, y)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] alpha=0.001, loss=log, max_iter=50, penalty=elasticnet, power_t=0.05, random_state=1337 


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.3s remaining:    0.0s


[CV]  alpha=0.001, loss=log, max_iter=50, penalty=elasticnet, power_t=0.05, random_state=1337, total=  54.2s
[CV] alpha=0.001, loss=log, max_iter=50, penalty=elasticnet, power_t=0.05, random_state=1337 


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.8min finished


[CV]  alpha=0.001, loss=log, max_iter=50, penalty=elasticnet, power_t=0.05, random_state=1337, total=  54.4s


GridSearchCV(cv=2, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.001], 'loss': ['log'], 'max_iter': [50], 'penalty': ['elasticnet'], 'power_t': [0.05], 'random_state': [1337]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=2)

In [22]:
print(clf0.best_score_)
print(clf0.best_params_)
# -14.8125892222
# {'alpha': 0.001, 'loss': 'log', 'max_iter': 250, 'random_state': 1337}

# parameters = {
#     'alpha': [0.001, 0.0001], 
#     'max_iter': [50,250,500],
#     'loss': ['log'],
#     'penalty': ['l2', 'l1', 'elasticnet'],
#     'power_t': [0.1, 0.3, 0.5],
#     'random_state': [1337],
# }
# -14.4717413019
# {'alpha': 0.001, 'loss': 'log', 'max_iter': 50, 'penalty': 'elasticnet', 'power_t': 0.1, 'random_state': 1337}
# In [14]:

# -14.4717413019
# {'alpha': 0.001, 'loss': 'log', 'max_iter': 50, 'penalty': 'elasticnet', 'power_t': 0.05, 'random_state': 1337}

-26.9960520848
{'alpha': 0.001, 'loss': 'log', 'max_iter': 50, 'penalty': 'elasticnet', 'power_t': 0.05, 'random_state': 1337}


In [14]:
test_preds = clf0.predict_proba(df_test)
print(test_preds)

[[ 0.    0.25  0.   ...,  0.    0.    0.25]
 [ 0.    0.25  0.   ...,  0.    0.    0.25]
 [ 0.    0.25  0.   ...,  0.    0.    0.25]
 ..., 
 [ 0.    0.25  0.   ...,  0.    0.    0.25]
 [ 0.    0.25  0.   ...,  0.    0.    0.25]
 [ 0.    0.25  0.   ...,  0.    0.    0.25]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [15]:
score2 = log_loss(test_labels, test_preds+1, labels = list(range(1,10)))
print('final multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('final f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('final accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

# earlier:
# parameters = {
#     'alpha': [0.001, 0.0001, 0.00001], 
#     'max_iter': [50,250,500,1000,5000],
#     'loss': ['log'],
#     'penalty': ['L2', 'L1', 'elasticnet'],
#     'power': [0.1, 0.03, 0.5]
#     'random_state': [1337],
# }
# final multi_log_loss: 21.11335970669253
# final f1_score: 0.2554347826086957
# final accuracy: 0.2554347826086957
# [[ 94   0   0   0   0   0   0   0   0]
#  [ 46   0   0   0   0   0   0   0   0]
#  [  7   0   0   0   0   0   0   0   0]
#  [ 65   0   0   0   0   0   0   0   0]
#  [ 25   0   0   0   0   0   0   0   0]
#  [ 22   0   0   0   0   0   0   0   0]
#  [101   0   0   0   0   0   0   0   0]
#  [  2   0   0   0   0   0   0   0   0]
#  [  6   0   0   0   0   0   0   0   0]]
# In [ ]:


final multi_log_loss: 2.197224577336219
final f1_score: 0.125
final accuracy: 0.125
[[  0  94   0   0   0   0   0   0   0]
 [  0  46   0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0   0   0]
 [  0  65   0   0   0   0   0   0   0]
 [  0  25   0   0   0   0   0   0   0]
 [  0  22   0   0   0   0   0   0   0]
 [  0 101   0   0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0]
 [  0   6   0   0   0   0   0   0   0]]


In [20]:
mlp_model = MLPClassifier()

# MLPClassifier(hidden_layer_sizes=(100, ), activation=’relu’, solver=’adam’, alpha=0.0001, 
#               batch_size=’auto’, learning_rate=’constant’, learning_rate_init=0.001, 
#               power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, 
#               verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
#               early_stopping=False, validation_fraction=0.1,
#               beta_1=0.9, beta_2=0.999, epsilon=1e-08)
parameters = {
    'solver': ['adam', 'sgd'], 
    'max_iter': [50,250,500],
    'learning_rate_init': [0.001, 0.0001, 0.01, 0.1]
    'alpha': [0.001],
    'power_t': [0.1],
    'random_state': [1337],
}

clf = GridSearchCV(mlp_model, parameters, n_jobs=4, 
                   cv=5,
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)



In [21]:
clf.fit(df_train, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV]  alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam, total=  22.8s
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV]  alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam, total=  26.3s
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=adam, total=  40.3s
[CV] alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=50, power_t=0.1, random_state=1337, solver=sgd, total=  19.0s
[CV] alpha=0.0001, max_iter=50, po

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  4.4min


[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd, total=  14.8s
[CV] alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=adam, total=  28.0s
[CV] alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd, total=  18.8s
[CV] alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd, total=  18.5s
[CV] alpha=0.0001, max_iter=250, power_t=0.3, random_state=1337, solver=adam 
[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=adam, total= 1.2min
[CV] alpha=0.0001, max_iter=250, power_t=0.3, random_state=1337, solver=adam 
[CV]  alpha=0.0001, max_iter=250, power_t=0.1, random_state=1337, solver=sgd, total=  17.8s
[CV] alpha=0.0001, max_iter=250, power_t=0.3, random_state=

[CV] alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=adam, total=  26.2s
[CV] alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd, total=  19.3s
[CV] alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd, total=  16.4s
[CV] alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=adam, total=  30.8s
[CV] alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd, total=  16.5s
[CV] alpha=0.001, max_iter=50, power_t=0.1, random_state=1337, solver=adam 
[CV]  alpha=0.0001, max_iter=500, power_t=0.5, random_state=1337, solver=sgd,

[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=adam 
[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=adam, total=  24.7s
[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd, total=  16.1s
[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd, total=  19.1s
[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=adam, total=  30.2s
[CV] alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=250, power_t=0.3, random_state=1337, solver=adam, total= 1.0min
[CV] alpha=0.001, max_iter=250, power_t=0.5, random_state=1337, solver=adam 
[CV]  al

[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 18.5min


[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=adam, total=  28.9s
[CV] alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd, total=  17.9s
[CV] alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd 
[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd, total=  22.9s
[CV] alpha=0.001, max_iter=500, power_t=0.3, random_state=1337, solver=adam 
[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=adam, total=  59.8s
[CV] alpha=0.001, max_iter=500, power_t=0.3, random_state=1337, solver=adam 
[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd, total=  21.0s
[CV] alpha=0.001, max_iter=500, power_t=0.3, random_state=1337, solver=adam 
[CV]  alpha=0.001, max_iter=500, power_t=0.1, random_state=1337, solver=sgd, total=  17.7s
[CV] alpha=0.001, max_iter=500, power_t=0.3, random_state=1337, solve

[Parallel(n_jobs=4)]: Done 180 out of 180 | elapsed: 21.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'solver': ['adam', 'sgd'], 'max_iter': [50, 250, 500], 'alpha': [0.0001, 0.001], 'power_t': [0.1, 0.3, 0.5], 'random_state': [1337]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=2)

In [24]:
print(clf.best_score_)
print(clf.best_params_)

# -2.19642899615
# {'alpha': 0.001, 'max_iter': 50, 'power_t': 0.1, 'random_state': 1337, 'solver': 'sgd'}

-2.19642899615
{'alpha': 0.001, 'max_iter': 50, 'power_t': 0.1, 'random_state': 1337, 'solver': 'sgd'}


In [22]:
test_preds = clf.predict_proba(df_test)
print(test_preds)

[[ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]
 [ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]
 [ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]
 ..., 
 [ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]
 [ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]
 [ 0.12718029  0.09598331  0.09304176 ...,  0.10990192  0.10837065
   0.1209434 ]]


In [32]:
print(test_preds.argmax(axis=1)+1)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [43]:
score2 = log_loss(test_labels, test_preds, labels = list(range(1,10)))
print('final multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='macro')
print('final f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('final accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))


final multi_log_loss: 2.197224577336219
final f1_score: 0.024691358024691357
final accuracy: 0.125
[[  0  94   0   0   0   0   0   0   0]
 [  0  46   0   0   0   0   0   0   0]
 [  0   7   0   0   0   0   0   0   0]
 [  0  65   0   0   0   0   0   0   0]
 [  0  25   0   0   0   0   0   0   0]
 [  0  22   0   0   0   0   0   0   0]
 [  0 101   0   0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0]
 [  0   6   0   0   0   0   0   0   0]]


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
